In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls

100epochs_3channel.h5  data_s_gen.h5		  Testdata_SigComp2011
data_1.h5	       data_siamese_forge.pickle  trainingSet
data.h5		       data_siamese_gen.pickle	  tranfer_20ep_l2+dropout.h5
data_s_forge1	       data_siamese.h5		  tranfer_20ep_l2reg.h5
data_s_forge1.h5       data_siamese.pickle	  tranfer_20ep_overfit.h5
data_s_gen1	       model.ipynb		  trial.ipynb
data_s_gen1.h5	       mycode.ipynb


In [0]:
import os 

In [0]:
os.chdir('drive/SOP')

In [0]:
import cv2
import numpy as np

In [0]:
forge = []

a = b = 0 
path = os.listdir('trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Forgeries/')
path.sort(key = lambda x: x[5:7])

for i in range(10):
  y = []
  for j in range(b,len(path)):
    if(path[j][5:7] == str(i+1).zfill(2)):
        img = cv2.imread('trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Forgeries/'+path[j]+'')
        img  = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        arr = np.asarray(img)
        arr = cv2.resize(arr,(1117,551),interpolation= cv2.INTER_AREA)
        y.append(arr)
        a = a + 1
        #print(a)
        if(a==235):
          forge.append(np.asarray(y))
    else:
     # print('...............')
      forge.append(np.asarray(y))
      b = a
      break

In [9]:
len(forge)

10

In [0]:
gen = []

a = b = 0 
path = os.listdir('trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Genuine/')
path.sort(key = lambda sample: sample[1:3])

for i in range(10):
  y = []
  for j in range(b,len(path)):
    if(path[j][1:3] == str(i+1).zfill(2)):
        img = cv2.imread('trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Genuine/'+path[j]+'')
        img  = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        arr = np.asarray(img)
        arr = cv2.resize(arr,(1117,551),interpolation= cv2.INTER_AREA)
        y.append(arr)
        a = a + 1
        #print(a)
        if(a==235):
          gen.append(np.asarray(y))
    else:
      #print('...............')
      gen.append(np.asarray(y))
      b = a
      break

In [0]:
X1 = []
X2 = []
X3 = []
for i in range(10):
  for j in range(len(gen[i])):
    for l in range(10):
      #y1 = []
      X1.append(gen[i][j])
      a = np.random.randint(0,len(gen[i])-1)
      X2.append(gen[i][a])
      k = np.random.randint(0,9)
      if(k == i):
        b = np.random.randint(0,len(forge[i])-1)
        X3.append(forge[i][b])
        #y1 = np.asarray(y1)
      else:
        b = np.random.randint(0,len(gen[k])-1)
        X3.append(gen[k][b])
        #y1 = np.asarray(y1)
      #x.append(y1)
X1 = np.asarray(X1).reshape((2350,551,1117,1))
X2 = np.asarray(X2).reshape((2350,551,1117,1))
X3 = np.asarray(X3).reshape((2350,551,1117,1))

In [0]:
Y = np.ones((2350,3,1024))
#Y2 = np.ones((2350,1,1,512))
#Y3 = np.ones((2350,1,1,512))

In [40]:
X1.shape

(2350, 551, 1117, 1)

order: pos, anchor, neg

In [0]:
base_model = keras.applications.nasnet.NASNetMobile(include_top=False, weights='imagenet', input_shape=(226,549,3))

In [0]:
base_model.summary()

In [13]:
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add,Reshape
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

Using TensorFlow backend.


In [0]:
def emb_model(input_shape):
  X_input = Input(input_shape)
  X = Conv2D(3,(21,21),strides=2,padding='valid')(X_input)
  #print(X.shape)
  X = base_model(X)
  X = MaxPooling2D((2,4))(X)
  X = Conv2D(256,(2,2),strides=2,padding='valid')(X)
  X = Flatten()(X)
  X = Reshape((1,1024))(X) 
  #print(X.shape)
  model = Model(inputs=X_input,outputs=X)
  return model

In [0]:
#X_input = Input((551,1117,1))
m = emb_model((551,1117,1))
for layer in base_model.layers:
    layer.trainable = False


In [42]:
 m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 551, 1117, 1)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 266, 549, 3)       1326      
_________________________________________________________________
NASNet (Model)               multiple                  4269716   
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 1056)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 2, 256)         1081600   
_________________________________________________________________
flatten_4 (Flatten)          (None, 1024)              0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 1, 1024)           0         
Total para

dont use this siam_model

In [32]:
#def siam_model(input_shape):
  X_input= Input((551,1117,3))
  #print(X_input[:,:,:,0])
  X1 = Input(tensor=Reshape((551,1117,1),input_shape=(551,1117))(X_input[:,:,:,0]))
  #print(X1.shape)
  X2 = Input(tensor=Reshape((551,1117,1),input_shape=(551,1117))(X_input[:,:,:,1]))
  X3 = Input(tensor=Reshape((551,1117,1),input_shape=(551,1117))(X_input[:,:,:,2]))

  X1 = Reshape((1,2048))(emb_model(X1))
  X2 = Reshape((1,2048))(emb_model(X2))
  X3 = Reshape((1,2048))(emb_model(X3))
  X = Concatenate(axis=0)([X1,X2,X3])
  #print(X.shape)
  #model = Model(inputs=X_input,outputs=X)
  #return model
  #model.summary()

Dont define a siam_model

In [0]:
  input_shape = (551,1117,1)
  X_input1= Input(input_shape)
  X_input2= Input(input_shape)
  X_input3= Input(input_shape)
  m = emb_model((551,1117,1))
  X11 = m(X_input1)
  X21 = m(X_input2)
  X31 = m(X_input3)
  X = Concatenate(axis=1)([X11,X21,X31])
  
  mod = Model(inputs=[X_input1,X_input2,X_input3],outputs=X)

In [0]:
import tensorflow as tf

In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[:,0,:], y_pred[:,1,:], y_pred[:,2,:]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(anchor - positive))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(anchor - negative))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = pos_dist - neg_dist + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(basic_loss, 0)
    ### END CODE HERE ###
    
    return loss

In [44]:
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 551, 1117, 1) 0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 551, 1117, 1) 0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 551, 1117, 1) 0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 1, 1024)      5352642     input_10[0][0]                   
                                                                 input_11[0][0]                   
          

In [0]:
mod.compile(optimizer='adam',loss=triplet_loss,metrics=['accuracy'])

In [0]:
mod.fit(x=[X1,X2,X3],y=Y,epochs=5,batch_size=1)

Epoch 1/5
2349/2350 [============================>.] - ETA: 0s - loss: 830.9069 - acc: 0.0021

2350/2350 [==============================] - 980s 417ms/step - loss: 830.5534 - acc: 0.0021
Epoch 2/5
 359/2350 [===>..........................] - ETA: 13:49 - loss: 432.0472 - acc: 0.0000e+00

2011/2350 [========================>.....] - ETA: 2:21 - loss: 500.7961 - acc: 0.0000e+00

In [0]:
 m.save('siamese_net.h5')

In [2]:
mod.evaluate(x=[X1,X2,X3],y=Y,batch_size=1)

NameError: ignored